In [2]:
import gym

In [3]:
env = gym.make('CartPole-v0')

In [4]:
env.reset()

array([-0.03912517,  0.03228889, -0.0026808 ,  0.01238816])

In [5]:
env.action_space

Discrete(2)

In [6]:
env.observation_space.shape[0]

4

In [7]:
env.reset()
for i in range(1000):
    random_action = env.action_space.sample()
    env.step(random_action)
    env.render()
env.close()

c:\program files\python36\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [10]:
for e in range(20):
    observation = env.reset()
    
    for t in range(50):
        env.render()
        action = env.action_space.sample()
        observation,reward,done,other_info = env.step(action)
        
        if done:
            # game episode is over
            print(f"Game episode:{e}/{20} High Score{t}")
            break
                  
env.close()
print('All 20 episodes are over')
            

Game episode:0/20 High Score9
Game episode:1/20 High Score18
Game episode:2/20 High Score14
Game episode:3/20 High Score13
Game episode:4/20 High Score12
Game episode:5/20 High Score20
Game episode:6/20 High Score10
Game episode:7/20 High Score13
Game episode:8/20 High Score28
Game episode:9/20 High Score20
Game episode:10/20 High Score12
Game episode:11/20 High Score35
Game episode:12/20 High Score29
Game episode:13/20 High Score17
Game episode:14/20 High Score11
Game episode:15/20 High Score11
Game episode:16/20 High Score10
Game episode:17/20 High Score37
Game episode:18/20 High Score11
Game episode:19/20 High Score21
All 20 episodes are over
